In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from datetime import timedelta, date
from dateutil.relativedelta import relativedelta as rd
import scipy.stats as stats
from copy import deepcopy

from tqdm import tqdm

In [2]:
# achatar startdate e enddate
def extrai_tempo(linha, **kwargs):
    valor = None    
    
    if kwargs['inicio'] == True:
        nome = 'startdate'
    else:
        nome = 'enddate'
    
    if kwargs['tipo'] == 'ano':
        pos_ini = 0
        pos_fim = 4
    else:
        pos_ini = 5
        pos_fim = 7
        
    try:
        valor = int(linha['employerorg_positionhistory_' + nome + '_anydate'][pos_ini:pos_fim])
    except:
        pass
    
    try:
        valor = int(linha['employerorg_positionhistory_' + nome + '_yearmonth'][pos_ini:pos_fim])
    except:
        pass
    
    if kwargs['tipo'] == 'ano':
        try: 
            valor = int(linha['employerorg_positionhistory_' + nome + '_year'])
        except:
            pass
    
    try:
        valor = int(valor)
    except:
        valor = None
    
    return valor

In [3]:
def intervalo_datas(data_inicio, data_fim):
    r = rd(data_fim, data_inicio)
    quantidade = r.years * 12 + r.months + 1
    for n in range(quantidade):
        yield data_inicio + rd(months=n)

In [4]:
MES_INICIO_MODA = 1
MES_FIM_MODA = 12
ANO_FIM_CORRENTE = 2018
MES_FIM_CORRENTE = 9
DATA_FIM_CORRENTE = date(ANO_FIM_CORRENTE, MES_FIM_CORRENTE, 1)

In [5]:
caminho = '../data/out_of_time/curriculosComInteresse/csv/'
# caminho = '../data/out_of_time/curriculosSemInteresse/csv/'

In [6]:
arquivos = [f for f in listdir(caminho) if isfile(join(caminho, f)) and '.csv' in f]

In [7]:
# Recuperando curriculos completos
for arquivo in arquivos:
    nome = arquivo.replace('.csv', '')
    exec(nome + ' = pd.read_csv(caminho + "' + arquivo + '", low_memory=False)')

In [8]:
dfs = [df for df in dir() if isinstance(eval(df), pd.DataFrame)]
dfs

['contactinfo',
 'culture',
 'employerorg',
 'experiencesummary',
 'language',
 'licenseorcertification',
 'personalinformation',
 'qualifications',
 'reference',
 'schoolorinstitution',
 'traininghistory']

In [9]:
employerorg['ano_inicio'] = employerorg.apply(extrai_tempo, axis=1, inicio=True, tipo='ano')
employerorg['mes_inicio'] = employerorg.apply(extrai_tempo, axis=1, inicio=True, tipo='mes')
employerorg['ano_fim'] = employerorg.apply(extrai_tempo, axis=1, inicio=False, tipo='ano')
employerorg['mes_fim'] = employerorg.apply(extrai_tempo, axis=1, inicio=False, tipo='mes')

In [10]:
ls = []
for i, l in employerorg.iterrows():
    try:
        dt_ini = date(int(l['ano_inicio']), int(l['mes_inicio']), 1)
        dt_fim = date(int(l['ano_fim']), int(l['mes_fim']), 1)
    
        if dt_ini > dt_fim:
            ls.append(l.id)
    except:
        if pd.isnull(l.ano_inicio) and pd.isnull(l.mes_inicio) and pd.isnull(l.ano_fim) and pd.isnull(l.mes_fim):
            ls.append(l.id)
    
employerorg = employerorg.loc[~employerorg.id.isin(ls)]
employerorg.reset_index(drop=True, inplace=True)

In [11]:
def seleciona_maior_data(mes_inicio, ano_inicio, mes_fim, ano_fim):
    try:
        mes_inicio = int(mes_inicio)
    except:
        mes_inicio = MES_INICIO_MODA
    
    try:
        mes_fim = int(mes_fim)
    except:
        mes_fim = MES_FIM_MODA
        
    try:
        data = date(int(ano_inicio), int(mes_inicio), 1)
    except ValueError:
        data = date(int(ano_fim), int(mes_fim), 1)
        
    return data

In [12]:
employerorg['data_ordem'] = employerorg.apply(lambda x : seleciona_maior_data(x.mes_inicio, x.ano_inicio, 
                                                                              x.mes_fim, x.ano_fim), axis=1)
employerorg.sort_values(by=['id', 'data_ordem'], ascending=False, inplace=True)

In [13]:
employerorg.head()

,employerorg_employerorgname,employerorg_positionhistory_description,employerorg_positionhistory_enddate_anydate,employerorg_positionhistory_enddate_stringdate,employerorg_positionhistory_enddate_year,employerorg_positionhistory_enddate_yearmonth,employerorg_positionhistory_jobcategory_categorycode,employerorg_positionhistory_jobcategory_comments,employerorg_positionhistory_jobcategory_taxonomyname,employerorg_positionhistory_orginfo_positionlocation_countrycode,...,employerorg_positionhistory_userarea_positionhistoryuserarea_skill_xhtml,employerorg_positionhistory_userarea_positionhistoryuserarea_skill_xml,employerorg_positionhistory_userarea_positionhistoryuserarea_subtitles_subtitle,employerorg_userarea_employerorguserarea_normalizedemployerorgname,id,ano_inicio,mes_inicio,ano_fim,mes_fim,data_ordem
9049,ordem dos advogados do brasil,* atuação na área de análise e desenvolvimento...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,cv9994.xml,2015.0,6.0,NaN,NaN,2015-06-01
9050,globaltec s/a,* atuação na área de análise e desenvolvimento...,NaN,NaN,NaN,2015-05-03:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,cv9994.xml,2014.0,7.0,2015.0,5.0,2014-07-01
9304,g&p projetos e sistemas ltda,principais atividades: instalação e configuraç...,NaN,NaN,NaN,2014-11-02:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,cv9991.xml,2013.0,2.0,2014.0,11.0,2013-02-01
9305,dataprev,principais atividades: auxílio nas tarefas exe...,NaN,NaN,NaN,2012-12-02:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,cv9991.xml,2012.0,2.0,2012.0,12.0,2012-02-01
711,sof - secretaria de orçamento federal,gestão estratégica da informação e respectivo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,cv9987.xml,2013.0,1.0,NaN,NaN,2013-01-01


In [14]:
guarda_id = None
employerorg['data_inicio'] = None
employerorg['data_fim'] =  None
for i, l in employerorg.iterrows():
    # guarda
    if guarda_id != l.id:
        guarda_id = l.id
        ano = ANO_FIM_CORRENTE
        mes = MES_FIM_CORRENTE
      
    try:
        ano_fim = int(l.ano_fim)
    except ValueError:
        ano_fim = int(ano)        

    try:
        mes_fim = int(l.mes_fim)
    except ValueError:
        mes_fim = int(mes)        
    
    ano = ano_fim
    mes = mes_fim
    employerorg.at[i, 'data_fim'] = date(ano, mes, 1)
    
    try: 
        ano_inicio = int(l.ano_inicio)
    except ValueError:
        ano_inicio = ano
        
    try:
        mes_inicio = int(l.mes_inicio)
    except ValueError:
        mes_inicio = MES_INICIO_MODA
    
    ano = ano_inicio
    mes = mes_inicio
    employerorg.at[i, 'data_inicio'] = date(ano, mes, 1)    

In [ ]:
ls = []

# recuperar os ids unicos encontrados no employerorg
ls_ids = np.unique(employerorg['id']).tolist()

guarda = {'id': None}

# iterar sobre os ids
for i in tqdm(ls_ids):
    
    temp = employerorg.loc[employerorg['id'] == i]
    
    # identificar data da primeira contratacao e data da ultima saida
    data_inicio = np.min(temp['data_inicio'])
    data_fim = np.max(temp['data_fim'])
    
    # lista para calcular tempo medio nos empregos
    ls_tempo_medio = []
    
    # iterar, mes a mes, criando um registro para cada id em cada mes
    for data in intervalo_datas(data_inicio, data_fim):
        if i == guarda['id']:
            linha = deepcopy(guarda)
        else:
            linha = {'id': i}
        
        
        # criar variavel alvo quando a data de fim for igual aa data iterada
        # incluir ou nao periodo desempregado na variavel alvo?
        if data in [x for x in temp['data_fim'].tolist() if x != DATA_FIM_CORRENTE]:
            linha['label'] = 1
        else:
            linha['label'] = 0
            
            
        #### obter detalhes da experiencia profissional        
        
        # experiencia total em meses
        try:
            linha['experiencia_meses'] += 1
        except KeyError:
            linha['experiencia_meses'] = 1
        
        
        # tempo no emprego em meses
        if data in temp['data_inicio'].tolist():
            linha['tempo_emprego'] = 1
        else:
            linha['tempo_emprego'] += 1
        
        
        # media de tempo de mudanca de emprego
        if linha['label'] == 1:
            ls_tempo_medio.append(linha['tempo_emprego'])
            linha['tempo_medio'] = np.mean(ls_tempo_medio)
        else:
            linha['tempo_medio'] = np.mean(ls_tempo_medio + [linha['tempo_emprego']])
        
        
        # interacao entre tempo no emprego atual e tempo medio nos empregos
        linha['taxa_permanencia'] = linha['tempo_emprego'] / np.float(linha['tempo_medio'])
        
        
        # tempo na empresa (mudanca de cargos)
        try:
            empresa = temp.loc[(temp['data_inicio'] <= data) & (temp['data_fim'] >= data), 
                               'employerorg_employerorgname'].values[0]
        except IndexError:
            empresa = None
            
        try:
            if linha['empresa'] == empresa:
                linha['tempo_empresa'] += 1
            else:
                linha['tempo_empresa'] = 1
        except KeyError:
            linha['tempo_empresa'] = 1
        
        linha['empresa'] = empresa
        
        
        # skills
        COMECO_CAMPO = 'employerorg_positionhistory_userarea_positionhistoryuserarea_skill_'
        skills = [x for x in temp.columns if COMECO_CAMPO in x]
        datas_menores = temp.loc[temp['data_fim'] <= data]
        for skill in skills:
            if datas_menores[skill].sum() > 0:
                s = skill.replace(COMECO_CAMPO, '').replace('#', 'sharp').replace('.', 'dot').replace('+', 'p')
                try:
                    linha['skill_' + s] += 1
                except KeyError:
                    linha['skill_' + s] = 1
        
        
        #
        
        CAMPO_CARGO = 'employerorg_positionhistory_userarea_positionhistoryuserarea_normalizedtitle'
        
        try:
            cargo = temp.loc[(temp['data_inicio'] <= data) & (temp['data_fim'] >= data), CAMPO_CARGO].values[0]
        except IndexError:
            cargo = None
            
        if pd.notnull(cargo):
            if 'gestor' in cargo or 'gerente' in cargo or 'coordenador' in cargo:
                try:
                    linha['cargo_gerente'] += 1
                except KeyError:
                    linha['cargo_gerente'] = 1
            else:
                linha['cargo_gerente'] = 0

            if 'analista' in cargo or 'programador' in cargo or 'desenvolvedor' in cargo:
                try:
                    linha['cargo_analista'] += 1
                except KeyError:
                    linha['cargo_analista'] = 1
            else:
                linha['cargo_analista'] = 0

            if 'consultor' in cargo:
                try:
                    linha['cargo_consultor'] += 1
                except KeyError:
                    linha['cargo_consultor'] = 1
            else:
                linha['cargo_consultor'] = 0

            if 'estagi' in cargo:
                try:
                    linha['cargo_estagiario'] += 1
                except KeyError:
                    linha['cargo_estagiario'] = 1
            else:
                linha['cargo_estagiario'] = 0

            if 'engenheir' in cargo:
                try:
                    linha['cargo_engenheiro'] += 1
                except KeyError:
                    linha['cargo_engenheiro'] = 1
            else:
                linha['cargo_engenheiro'] = 0

        #
        
#         CAMPO_AUTONOMO = 'employerorg_positionhistory_userarea_positionhistoryuserarea_isselfemployed'

#         if 'trabalho_autonomo' not in linha:
#             try:
#                 autonomo = temp.loc[(temp['data_inicio'] <= data) & (temp['data_fim'] >= data), 
#                                         CAMPO_AUTONOMO].values[0]
#             except IndexError:
#                 autonomo = None

#             if pd.notnull(autonomo):
#                 linha['trabalho_autonomo'] = 1
#             else:
#                 linha['trabalho_autonomo'] = 0

                
        # tempo no mesmo cargo        
        try:
            if linha['cargo'] == cargo:
                linha['tempo_cargo'] += 1
            else:
                linha['tempo_cargo'] = 1
        except KeyError:
            linha['tempo_cargo'] = 1
            
        linha['cargo'] = cargo
        
        
        # mudanca de localidade 
        CAMPO_MUNICIPIO = 'employerorg_positionhistory_orginfo_positionlocation_municipality'
        
        try:
            municipio = temp.loc[(temp['data_inicio'] <= data) & (temp['data_fim'] >= data), 
                                 CAMPO_MUNICIPIO].values[0]
        except IndexError:
            municipio = None  
            
        try:
            if linha['municipio_empresa'] == municipio:
                linha['tempo_municipio'] +=1
            else:
                linha['tempo_municipio'] = 1
        except KeyError:
            linha['tempo_municipio'] = 1
        
        linha['municipio_empresa'] = municipio
        
        
        #### acrescentar campos a partir das outras tabelas
        
        # faculdade
#         CURSO_CAMPO = 'schoolorinstitution_degree_degreemajor_name'
#         NIVEL_CAMPO = 'schoolorinstitution_degree_userarea_degreeuserarea_normalizeddegreetype'

#         datas_menores = schoolorinstitution.loc[(schoolorinstitution['id'] == i) & 
#                                                 (schoolorinstitution['data_fim'] <= data)]

#         for j, cursos in datas_menores.iterrows():
#             if cursos[NIVEL_CAMPO] in ['bachelors', 'bsc', 'doctorate', 'masters', 
#                                                  'mba', 'msc', 'some college']: 
                
#                 if pd.notnull(cursos[CURSO_CAMPO]):
#                     if 'administracao' in cursos[CURSO_CAMPO] or 'gest' in cursos[CURSO_CAMPO] or \
#                        'geren' in cursos[CURSO_CAMPO]:
#                         try:
#                             linha['curso_administracao'] += 1
#                         except KeyError:
#                             linha['curso_administracao'] = 1

#                     if 'comput' in cursos[CURSO_CAMPO] or 'desenv' in cursos[CURSO_CAMPO] or \
#                        'inform' in cursos[CURSO_CAMPO]:
#                         try:
#                             linha['curso_computacao'] += 1
#                         except KeyError:
#                             linha['curso_computacao'] = 1

#                     if 'engenh' in cursos[CURSO_CAMPO]:
#                         try:
#                             linha['curso_engenharia'] += 1
#                         except KeyError:
#                             linha['curso_engenharia'] = 1
        
#         ##
 
#         try:
#             localizacao = contactinfo.loc[contactinfo['id'] == i, 
#                                           'contactinfo_contactmethod_postaladdress_region'].values[0]
#         except IndexError:
#             localizacao = None
        
#         locs = [x for x in linha.keys() if 'loc-' in x]
#         for lc in locs:
#             linha[lc] = 0
        
#         if pd.notnull(localizacao):
#             linha['loc-' + localizacao] = 1
#         else:
#             linha['loc-br-df'] = 1
        
#         ##
#         try:
#             pais_origem = culture.loc[culture['id'] == i,
#                                       'culture_country'].values[0]
#         except IndexError:
#             pais_origem = None
            
#         if pd.notnull(pais_origem):
#             linha['pais-origem-' + pais_origem] = 1
#         else:
#             linha['pais-origem-br'] = 1
                    
#         #
#         try:
#             idioma_nativo = culture.loc[culture['id'] == i,
#                                         'culture_language'].values[0]
#         except IndexError:
#             idioma_nativo = None
            
#         if pd.notnull(idioma_nativo):
#             linha['idioma-nativo-' + idioma_nativo] = 1
#         else:
#             linha['idioma-nativo-pt'] = 1
        
        ##
        
        if 'idioma-total' not in linha:
            idiomas = language.loc[language['id'] == i]
            linha['idioma-total'] = len(idiomas)

            for index, idioma in idiomas.iterrows():
                nome_idioma = idioma['language_languagecode']

                if idioma['language_read'] == True:
                    linha['idioma-' + nome_idioma + '-le'] = 1
                if idioma['language_speak'] == True:
                    linha['idioma-' + nome_idioma + '-fala'] = 1
                if idioma['language_write'] == True:
                    linha['idioma-' + nome_idioma + '-escreve'] = 1

        #
        if 'homem' not in linha and 'mulher' not in linha:
            try:
                sexo = personalinformation.loc[personalinformation['id'] == i,
                                               'personalinformation_gender'].values[0]
            except IndexError:
                sexo = None
                
            if pd.notnull(sexo):
                if sexo == 'male':
                    linha['homem'] = 1

                if sexo == 'female':
                    linha['mulher'] = 1                
            else:
                linha['homem'] = 0
                linha['mulher'] = 0
            
        #
        
        if 'solteiro' not in linha and 'casado' not in linha:
            try:
                estado_civil = personalinformation.loc[personalinformation['id'] == i,
                                                       'personalinformation_maritalstatus'].values[0]
            except IndexError:
                estado_civil = None
            
            if pd.isnull(estado_civil):
                estado_civil = ''
                
            if estado_civil == 'single' or estado_civil == 'separated' or estado_civil == 'divorced':
                linha['solteiro'] = 1
            else:
                linha['solteiro'] = 0

            if estado_civil == 'married':
                linha['casado'] = 1
            else:
                linha['casado'] = 0                    
        #
        
#         if 'idade' not in linha:
#             try:
#                 data_nascimento = personalinformation.loc[personalinformation['id'] == i,
#                                                           'personalinformation_dateofbirth'].values[0]
#             except IndexError:
#                 data_nascimento = None
                
#             if pd.notnull(data_nascimento):
#                 dt_nsc = date(int(data_nascimento[0:4]), 
#                               int(data_nascimento[5:7]), 
#                               int(data_nascimento[8:10]))
#                 r = rd(data, dt_nsc)
#                 idade = r.years * 12 + r.months + 1
#                 linha['idade'] = idade
#             else:
#                 linha['idade'] = 0
#         else:
#             if linha['idade'] > 0:
#                 linha['idade'] += 1
        
        #
        
        if 'tem_cnh' not in linha:
            try:
                tem_cnh = personalinformation.loc[personalinformation['id'] == i,
                                              'personalinformation_drivinglicense'].values[0]
            except IndexError:
                tem_cnh = None
                
            if pd.notnull(tem_cnh):
                linha['tem_cnh'] = 1
            else:
                linha['tem_cnh'] = 0
        
        # 
        
#         if 'sal_pretendido' not in linha:
#             try:
#                 salario_pretendido = personalinformation.loc[personalinformation['id'] == i, 
#                                                              'personalinformation_requiredsalary'].values[0]
#             except IndexError:
#                 salario_pretendido = None

#             if pd.notnull(salario_pretendido):
#                 linha['sal_pretendido'] = salario_pretendido
#             else:
#                 linha['sal_pretendido'] = 0
        
#         #
        
#         if 'tem_familia' not in linha:
#             try:
#                 tem_familia = personalinformation.loc[personalinformation['id'] == i, 
#                                                       'personalinformation_familycomposition'].values[0]
#             except IndexError:
#                 tem_familia = None

#             if pd.notnull(tem_familia):
#                 linha['tem_familia'] = 1
#             else:
#                 linha['tem_familia'] = 0

#         #
        
#         if 'tem_lugar_nasc' in linha:
#             try:
#                 lugar_nasc = personalinformation.loc[personalinformation['id'] == i,
#                                                     'personalinformation_birthplace'].values[0]
#             except IndexError:
#                 lugar_nasc = None

#             if pd.notnull(lugar_nasc):
#                 linha['tem_lugar_nasc'] = 1
#             else:
#                 linha['tem_lugar_nasc'] = 0
        
#         #
        
#         if 'tem_observacoes' in linha:
#             try:
#                 tem_observacoes = experiencesummary.loc[experiencesummary['id'] == id, 
#                                                         'experiencesummary_attentionneeded'].values[0]
#             except IndexError:
#                 tem_observacoes = None

#             if pd.notnull(tem_observacoes):
#                 linha['tem_observacoes'] = 1
#             else:
#                 linha['tem_observacoes'] = 0
        
#         #
        
#         if 'tem_website' in linha:
#             try:
#                 tem_website = contactinfo.loc[contactinfo['id'] == i,
#                                              'contactinfo_contactmethod_internetwebaddress'].values[0]
#             except IndexError:
#                 tem_website = None

#             if pd.notnull(tem_website):
#                 linha['tem_website'] = 1
#             else:
#                 linha['tem_website'] = 0
        
#         #
        
#         if 'tem_qualificacoes' in linha:
#             try:
#                 tem_qualificacoes = qualifications.loc[qualifications['id'] == i, 'id'].values[0]
#             except IndexError:
#                 tem_qualificacoes = None
                
#             if pd.notnull(tem_qualificacoes):
#                 linha['tem_qualificacoes'] = 1
#             else:
#                 linha['tem_qualificacoes'] = 0
        
#         #
        
#         if 'tem_referencias' in linha:
#             try:
#                 tem_referencias = reference.loc[reference['id'] == i, 'id'].values[0]
#             except IndexError:
#                 tem_referencias = None

#             if pd.notnull(tem_referencias):
#                 linha['tem_referencias'] = 1
#             else:
#                 linha['tem_referencias'] = 0                
        
        # manter guarda atualizada        
        guarda = deepcopy(linha)
        
        # criar linha
        ls.append(linha)

  1%|          | 32/2743 [03:25<6:01:04,  7.99s/it]

In [ ]:
# criar dataframe 
df = pd.DataFrame(ls)

In [ ]:
dict(df.count())

In [ ]:
df

In [ ]:
df = df.fillna(0)
df

In [ ]:
df.to_csv(caminho + 'df.csv', index=False)